In [ ]:
import torch
import torch.nn as nn
from models import *

# Assuming the quantization functions and classes are defined as provided earlier

# Load the model
model = resnet20_cifar().to('cuda')
model_name = 'resnet20_cifar'
fdir = 'result/' + str(model_name) + '/model_best.pth.tar'
checkpoint = torch.load(fdir)
model.load_state_dict(checkpoint['state_dict'])

# Quantize weights
def quantize_weights(model, w_bits=4):
    state_dict = model.state_dict()
    for name, param in state_dict.items():
        if 'conv' in name and 'weight' in name:
            # Quantize the weights to w_bits
            wgt_alpha = max(param.abs())
            weight_quant = weight_quantization(w_bits)
            state_dict[name] = weight_quant(param, wgt_alpha).mul(wgt_alpha)
    model.load_state_dict(state_dict)

# Apply the quantization function to the model's weights
quantize_weights(model, w_bits=4)  # For 4-bit quantization

# Now the model's weights are quantized and the model is ready to be evaluated.
